<a href="https://colab.research.google.com/github/ousdiallo/Reinforcement-Learning/blob/main/Rendering_OpenAi_Gym_in_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install dependancies, takes around 45 seconds

Rendering Dependancies



In [2]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [4]:
!apt update && apt install xvfb && pip3 install pyvirtualdisplay && pip install pyvirtualdisplay

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

Pacman Dependancies

In [1]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

# Imports and Helper functions


In [2]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [3]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [4]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# Env Enquires

In [5]:
import gym

def query_environment(name):
  env = gym.make(name)
  spec = gym.spec(name)
  print(f"Action Space: {env.action_space}")
  print(f"Observation Space: {env.observation_space}")
  print(f"Max Episode Steps: {spec.max_episode_steps}")
  print(f"Nondeterministic: {spec.nondeterministic}")
  print(f"Reward Range: {env.reward_range}")
  print(f"Reward Threshold: {spec.reward_threshold}")

In [6]:
query_environment("CartPole-v1")

Action Space: Discrete(2)
Observation Space: Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)
Max Episode Steps: 500
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: 475.0


# CartPole!

In [ ]:
env = wrap_env(gym.make("CartPole-v1"))

In [ ]:
#check out the action space!
print(env.action_space)

In [ ]:
observation = env.reset()

while True:
  
    env.render()
    
    #your agent goes here
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action) 
   
        
    if done: 
      break;
            
env.close()
show_video()

# Test

In [7]:
import numpy as np

# Valeurs hautes et basses des observations
low_values=np.array([-5, -5, -0.45, -5])
high_values=np.array([5, 5, 0.45, 5])

division=[42, 42, 42, 42]
pas=(high_values-low_values)/division

def discretise(state):
    discrete_state=(state-low_values)/pas
    return tuple(discrete_state.astype(np.int))


In [10]:
import gym
import numpy as np
import cv2
import CartPole_common

env=gym.make("CartPole-v0")
env._max_episode_steps=500

alpha=0.05
gamma=0.98

epoch=50000
show_every=500

epsilon=1.
epsilon_min=0.05
start_epsilon=1
end_epsilon=epoch//2
epsilon_decay_value=epsilon/(end_epsilon-start_epsilon)

nbr_action=env.action_space.n
q_table=np.random.uniform(low=-1, high=1, size=(CartPole_common.division+[nbr_action]))

result_done=0
scores=[]
best_score=0
for episode in range(epoch):
    obs=env.reset()
    discrete_state=CartPole_common.discretise(obs)
    done=False

    if episode%show_every == 0:
        render=True
        mean_score=np.mean(scores)
        print("Epoch {:06d}/{:06d} reussite:{:04d}/{:04d} epsilon={:06.4f} Mean score={:08.4f} alpha={:06.4f}".format(episode, epoch, result_done, show_every, epsilon, mean_score, alpha))
        scores=[]
        result_done=0
        if mean_score>best_score:
            print("Sauvegarde ...")
            np.save("CartPole_qtable", q_table)
            best_score=mean_score
        alpha=alpha*0.99

    else:
        render=False

    score=1
    while not done:

        if np.random.random()>epsilon:
            action=np.argmax(q_table[discrete_state])
        else:
            action=np.random.randint(nbr_action)

        new_state, reward, done, info=env.step(action)
        new_discrete_state=CartPole_common.discretise(new_state)

        if episode%show_every == 0:
            env.render()

        #reward=2-np.abs(new_state[0])
        if done:
            scores.append(score)
            if score==env._max_episode_steps:
                result_done+=1
            else:
                reward=-10

        max_future_q=np.max(q_table[new_discrete_state])
        current_q=q_table[discrete_state][action]
        new_q=(1-alpha)*current_q+alpha*(reward+gamma*max_future_q)
        q_table[discrete_state][action]=new_q

        score+=1
        discrete_state=new_discrete_state

    if end_epsilon>=episode>=start_epsilon:
        epsilon-=epsilon_decay_value
        if epsilon<epsilon_min:
            epsilon=epsilon_min

env.close()


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 000000/050000 reussite:0000/0500 epsilon=1.0000 Mean score=00000nan alpha=0.0500
Epoch 000500/050000 reussite:0000/0500 epsilon=0.9800 Mean score=022.7120 alpha=0.0495
Sauvegarde ...
Epoch 001000/050000 reussite:0000/0500 epsilon=0.9600 Mean score=022.9880 alpha=0.0490
Sauvegarde ...
Epoch 001500/050000 reussite:0000/0500 epsilon=0.9400 Mean score=021.9140 alpha=0.0485
Epoch 002000/050000 reussite:0000/0500 epsilon=0.9200 Mean score=023.6000 alpha=0.0480
Sauvegarde ...
Epoch 002500/050000 reussite:0000/0500 epsilon=0.9000 Mean score=023.3800 alpha=0.0475
Epoch 003000/050000 reussite:0000/0500 epsilon=0.8800 Mean score=023.4020 alpha=0.0471
Epoch 003500/050000 reussite:0000/0500 epsilon=0.8600 Mean score=025.6940 alpha=0.0466
Sauvegarde ...
Epoch 004000/050000 reussite:0000/0500 epsilon=0.8400 Mean score=025.1160 alpha=0.0461
Epoch 004500/050000 reussite:0000/0500 epsilon=0.8200 Mean score=026.8780 alpha=0.0457
Sauvegarde ...
Epoch 005000/050000 reussite:0000/0500 epsilon=0.8000 M

In [ ]:
import gym
import numpy as np
import CartPole_common

env=gym.make("CartPole-v0")
env._max_episode_steps=5000

q_table=np.load("CartPole_qtable.npy")

for epoch in range(1000):
    state = env.reset()
    score = 0
    while True:
        env.render()
        discrete_state=CartPole_common.discretise(state)
        action=np.argmax(q_table[discrete_state])
        #if not np.random.randint(5):
        #    action=np.random.randint(2)
        state, reward, done, info=env.step(action)
        score+=reward
        if done:
            print('Essai {:05d} Score: {:04d}'.format(epoch, int(score)))
            break

env.close()


Essai 00000 Score: 0875
Essai 00001 Score: 5000
Essai 00002 Score: 5000
Essai 00003 Score: 5000
Essai 00004 Score: 5000
Essai 00005 Score: 1160
Essai 00006 Score: 5000
Essai 00007 Score: 0458
Essai 00008 Score: 5000
Essai 00009 Score: 5000
Essai 00010 Score: 5000
Essai 00011 Score: 1067
